In [4]:
import os

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import pancancer_evaluation.config as cfg
import pancancer_evaluation.utilities.analysis_utilities as au

In [5]:
stratified_results_dir = os.path.join(cfg.repo_root, 'stratified_results', 'pancancer')

In [9]:
vogelstein_df = (
    au.load_prediction_results(stratified_results_dir, 'stratified')
      .assign(gene_dataset='vogelstein')
      .drop(columns=['holdout_cancer_type'])
)
vogelstein_df.head(n=10)

,auroc,aupr,gene,signal,seed,data_type,fold,train_set,identifier,gene_dataset
0,0.95513,0.70461,MAP3K1,signal,42,train,0,stratified,NaN,vogelstein
1,0.69504,0.32503,MAP3K1,signal,42,test,0,stratified,NaN,vogelstein
2,0.68868,0.29750,MAP3K1,signal,42,cv,0,stratified,NaN,vogelstein
3,0.98730,0.90617,MAP3K1,signal,42,train,1,stratified,NaN,vogelstein
4,0.78217,0.42547,MAP3K1,signal,42,test,1,stratified,NaN,vogelstein
5,0.68345,0.22054,MAP3K1,signal,42,cv,1,stratified,NaN,vogelstein
6,0.99257,0.91245,MAP3K1,signal,42,train,2,stratified,NaN,vogelstein
7,0.56816,0.23178,MAP3K1,signal,42,test,2,stratified,NaN,vogelstein
8,0.69348,0.33192,MAP3K1,signal,42,cv,2,stratified,NaN,vogelstein
9,0.99455,0.93297,MAP3K1,signal,42,train,3,stratified,NaN,vogelstein
